In [1]:
import pandas as pd
# import scprep as sc
import scanpy as sc
import phate
import numpy as np
import seaborn as sns
import scprep
from sklearn.metrics import roc_auc_score, average_precision_score
import os
from tqdm import tqdm
import warnings


In [2]:
from tqdm import tqdm
import warnings
from statsmodels.tsa.stattools import grangercausalitytests

lag_order = 1 # since we aggregated the data in to 9 bins we only need 1 lag
maxlag = (
    lag_order,  # becuase we got this value before. We are not suppose to add 1 to it
)
test = "ssr_chi2test"

from joblib import Parallel, delayed

def grangers_causation_matrix(
    data, in_variables, out_variables, test="ssr_chi2test", n_jobs=1, warn=False
):
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """

    def get_pval(dd):
        if warn:
            test_result = grangercausalitytests(dd, maxlag=maxlag, verbose=True)
        else:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=FutureWarning)
                test_result = grangercausalitytests(dd, maxlag=maxlag, verbose=False)
                # according to the documentation https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.grangercausalitytests.html,
                # the dd has 2 columns, second causes the first.
                # print(test_result.keys())
                # print(f'dd shape {dd.shape}')
                # print(test_result[1][1][0].params[0])

                # print(test_result[1][1][0].summary())
                # print(test_result[1][1][1].summary())
                # print(test_result[1][1][1].model.exog_names)

                # print(test_result[2][1][1].summary())
                


                # print(test_result[1][1][0].model.params)
                # print(test_result[1])
                # print(test_result[1][1][0].params)
                # print(test_result[1][1][0].params['constant'])
                # print(test_result[1][1][0].model.exog_names)
        # import pdb; pdb.set_trace()
        # print(dd.columns)
        # print(dd.head())
        # print(test_result[1][1][1].params)
        p_values = [test_result[i][0][test][1] for i in maxlag] # test_result[i][1] is the unrestricted model, test_result[i][1][0] is the restricted model
        # coefs = [test_result[i][1][1].params[0] for i in maxlag]
        coefs = [test_result[i][1][1].params[1] for i in maxlag] # x1, x2, const

        arg_min_p_value = np.argmin(p_values)
        min_p_value = p_values[arg_min_p_value]
        min_coef = coefs[arg_min_p_value]
        # print(p_values)
        # print(coefs)
        return (min_p_value, min_coef)


    out = Parallel(n_jobs=n_jobs)(
        # delayed(get_pval)(data[[r, c]]) # this is incorrect
        delayed(get_pval)(data[[c, r]]) # this means r causes c, so r is be in and c is out
        for c in tqdm(out_variables, desc="Processing columns")  # Outer loop progress bar
        for r in tqdm(in_variables, desc="Processing rows", leave=False)  # Inner loop progress bar
    )
    # Note that this is the wrong way and must be corrected
    # df = pd.DataFrame(
    #     np.array(out).reshape((len(in_variables), len(out_variables))), # this is incorrect
    #     columns=out_variables,
    #     index=in_variables,
    # )
    out_p = [p for (p,c) in out]
    out_c = [c for (p,c) in out]
    df_p = pd.DataFrame(
        np.array(out_p).reshape((len(out_variables), len(in_variables))), # should be reshaped to len(out_variables), len(in_variables) according to the for loop.
        columns=in_variables,
        index=out_variables,
    ).T # used the correct reshaping, and then transposed the matrix so the x and y are semantically correct (x causes y).
    df_c = pd.DataFrame(
        np.array(out_c).reshape((len(out_variables), len(in_variables))), # should be reshaped to len(out_variables), len(in_variables) according to the for loop.
        columns=in_variables,
        index=out_variables,
    ).T
    df_p.index = [var + "_x" for var in in_variables]
    df_p.columns = [var + "_y" for var in out_variables]
    df_c.index = [var + "_x" for var in in_variables]
    df_c.columns = [var + "_y" for var in out_variables]
    return df_p, df_c

def do_granger(trajs, in_genes, out_genes, n_jobs=1, warn=False):
    # in causes out
    trajs = trajs.T[::10]
    trajs = trajs - trajs.shift(1)
    trajs = trajs.dropna()
    out_traj_p, out_traj_c = grangers_causation_matrix(
        trajs, in_variables=in_genes, out_variables=out_genes, n_jobs=n_jobs, warn=warn
    )
    return out_traj_p, out_traj_c



In [3]:
# traj_gene_sp = np.load("../data_human_samples/10-trajectories_gene_space.npy")
traj_gene_sp = np.load(f"../results_final_Dec22/traj_gene_space_A_extreme.npy", allow_pickle=True)

In [4]:
adata = sc.read_h5ad('../data/rna_figure_ready.h5ad')

In [5]:
# sc.pp.highly_variable_genes(adata, n_top_genes=400)
alex_genes = open('../data/alex_genes.txt').read().splitlines()
alex_tfs = open('../data/alex_tfs.txt').read().splitlines()

In [6]:
# gene_mask = np.isin(adata.var_names, alex_genes)
# tf_mask = np.isin(adata.var_names, alex_tfs)

In [7]:
alex_gene_all = list(set(alex_genes) | set(alex_tfs))
alex_gene_all_mask = np.isin(adata.var_names, alex_gene_all)
traj_subset = traj_gene_sp[:, :, alex_gene_all_mask]
gene_names = adata.var_names[alex_gene_all_mask]

In [8]:
traj_subset.shape

(100, 19, 2226)

In [9]:
# traj_df = pd.DataFrame(np.transpose(traj_subset, (1, 0, 2)).mean(axis=0), columns=gene_names)
data = np.transpose(traj_subset, (1, 0, 2))
valid_mask = (data.mean(axis=0).var(axis=0) != 0.0)
data = data[:, :, valid_mask]
gene_names = np.array(gene_names)[valid_mask]
traj_df = pd.DataFrame(data.mean(axis=0), columns=gene_names)

In [23]:
gm_p, gm_c = do_granger(traj_df.T, in_genes=alex_tfs, out_genes=alex_genes, n_jobs=-1)

KeyboardInterrupt: 

In [25]:
gm_p.to_csv(f'../results_final_Jan15/granger_A_extreme_p.csv')
gm_c.to_csv(f'../results_final_Jan15/granger_A_extreme_c.csv')

In [24]:
gm_p

,NOC2L_y,HES4_y,ISG15_y,ERRFI1_y,ENO1_y,PGD_y,CENPS_y,SRM_y,PLOD1_y,EFHD2_y,...,MT-ATP6_y,MT-CO3_y,MT-ND3_y,MT-ND4L_y,MT-ND4_y,MT-ND5_y,MT-ND6_y,MT-CYB_y,CDH1_y,GAPDH_y
HES4_x,7.794867e-01,1.000000,8.372976e-01,0.116870,6.900046e-01,0.278783,1.619419e-01,0.504819,0.529470,0.024509,...,0.538011,0.828883,0.190035,0.096815,0.545869,0.456374,0.489247,0.628664,0.682319,0.241362
CENPS_x,5.928597e-01,0.263094,8.143832e-01,0.286377,1.451621e-01,0.243627,1.000000e+00,0.289895,0.251571,0.668853,...,0.543833,0.986521,0.094266,0.002208,0.473953,0.903837,0.056136,0.626530,0.926442,0.633049
RUNX3_x,2.582658e-01,0.915070,1.839757e-01,0.324194,8.304607e-01,0.679616,3.086926e-01,0.357884,0.635528,0.861176,...,0.551438,0.303496,0.427541,0.433989,0.587324,0.368795,0.550797,0.425481,0.274973,0.330547
AHDC1_x,6.298774e-02,0.124101,4.313915e-01,0.289074,9.371241e-01,0.576335,7.290990e-01,0.563217,0.003036,0.026485,...,0.190017,0.037191,0.442577,0.317477,0.272262,0.014423,0.047572,0.105048,0.739353,0.883073
ZBTB8A_x,6.000910e-02,0.151064,4.340566e-01,0.465856,7.560912e-01,0.043874,1.294319e-02,0.749767,0.002441,0.000794,...,0.356983,0.105181,0.796607,0.539418,0.085120,0.097628,0.150561,0.305751,0.569843,0.004457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BACH1_x,8.073398e-14,0.765237,6.599131e-02,0.794206,9.074666e-07,0.000228,3.409410e-01,0.677474,0.069198,0.271205,...,0.025187,0.008767,0.741972,0.842292,0.000201,0.006456,0.233957,0.034824,0.455075,0.059667
SON_x,3.157318e-01,0.810677,1.684070e-07,0.680929,5.349956e-01,0.137970,3.263248e-10,0.062316,0.161871,0.005133,...,0.188457,0.045914,0.417082,0.474192,0.112577,0.009697,0.085100,0.143120,0.993611,0.000006
RUNX1_x,4.220905e-02,0.735611,2.187768e-02,0.077887,1.289822e-01,0.426733,2.897092e-01,0.578925,0.216537,0.764961,...,0.098459,0.006938,0.514078,0.389230,0.035763,0.013333,0.418122,0.063549,0.646476,0.109033
ETS2_x,4.537645e-02,0.804256,3.202202e-03,0.378814,5.180735e-01,0.839500,4.776031e-02,0.242256,0.020652,0.545955,...,0.272893,0.042190,0.249432,0.124266,0.198674,0.023449,0.029340,0.191060,0.795255,0.012389


In [14]:
# log_pval_df = -np.log10(pval_df)
log_pval_df = -np.log(gm_p + 2 ** -10)
coef_sign_df = np.sign(gm_c)
signed_score_df = log_pval_df * coef_sign_df

In [16]:
signed_score_df.columns = [c.strip('_y') for c in signed_score_df.columns]
signed_score_df.index = [r.strip('_x') for r in signed_score_df.index]

In [26]:
signed_score_df.to_csv(f'../results_final_Jan15/granger_A_extreme_signed_score.csv')

In [20]:
'ESRRA' in signed_score_df.columns

True

In [21]:
'ESRRA' in signed_score_df.index

False